In [1]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from IPython import get_ipython
import os
import base64
from pyngrok import ngrok
import random
import eye_tracking
import voice
import voice_output

In [2]:
# pip install import-ipynb
# import import_ipynb
# from "../voice" import voice.ipynb
# %run ../voice/voice.ipynb
# import sys
# sys.path.append("../voice")
# sys.path.append("../video")
# sys.path.append("../eye_track")
# from eye_track import eye_tracking as eye_main




In [3]:
app = Flask(__name__)
CORS(app)

In [4]:
# Set the path to store videos
UPLOAD_FOLDER = '/home/jegathees5555/Documents/recruitz/backend/video/'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

In [5]:
def upload_video(video_data):        
   

    if video_data:
        video_name = 'uploaded_video.webm'
        video_path = os.path.join(app.config['UPLOAD_FOLDER'], video_name)
        print(video_path)
        print(video_name)
        video_data.save(video_path)  # Save the file
        
        
        
        # return result
        return jsonify({'message': 'Video uploaded successfully'}), 200
    else:
        return jsonify({'error': 'No video data provided'}), 400

In [6]:
def eye_track_module():
    # Run the other Jupyter notebook with the correct path
        eye_tracking_notebook_path = '/home/jegathees5555/Documents/recruitz/backend/eye_track/EyeTracking.ipynb'
        # get_ipython().run_line_magic('run', eye_tracking_notebook_path)

        result = eye_tracking.eye_tracking_method('/home/jegathees5555/Documents/recruitz/backend/video/uploaded_video.webm')
        print(result)
        return result

In [7]:
# from backend.eye_track.eye_tracking import eye_tracking


@app.route('/upload_video_new', methods=['POST'])
def upload_frontend_video():
    try:
         # Remove the assumption of JSON data
        video_data = request.files['videoData']
        upload_video(video_data)
    except Exception as e:
        return jsonify({'error': str(e)}), 500
    
    try:
        result = eye_track_module()
    except Exception as e:
        return jsonify({'error': str(e)}), 500
    
    result_str = str(result)

    return jsonify({'message': 'Video uploaded successfully', 'result': result_str}), 200
    

In [8]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the pre-trained FER model
model = load_model("/home/jegathees5555/Documents/recruitz/backend/server/FER_model.h5")  # You need to have the model file (fer_model.h5) available

def analyze_boldness_and_confidence():
    video_path = '/home/jegathees5555/Documents/recruitz/backend/video/uploaded_video.webm'
    face_cascade = cv2.CascadeClassifier('/home/jegathees5555/Documents/recruitz/backend/server/haarcascade_frontalface_default.xml')

    video_capture = cv2.VideoCapture(video_path)

    confidence_scores = []

    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in faces:
            face_roi = gray_frame[y:y + h, x:x + w]
            resized_roi = cv2.resize(face_roi, (48, 48))
            normalized_roi = resized_roi / 255.0
            reshaped_roi = np.reshape(normalized_roi, (1, 48, 48, 1))
            emotion_predictions = model.predict(reshaped_roi)
            confidence_score = emotion_predictions[0][3]  # Confidence score for 'happy' emotion

            if 0.75 <= confidence_score < 0.90:
                emotion = "happy"
            elif 0.50 <= confidence_score < 0.75:
                emotion = "neutral"
            else:
                emotion = "sad"

            confidence_scores.append(confidence_score)

    video_capture.release()

    average_confidence = sum(confidence_scores) / len(confidence_scores)
    return average_confidence * 1300 

2023-08-27 15:06:10.367998: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-27 15:06:10.411972: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-27 15:06:10.699617: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-27 15:06:10.701466: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-27 15:06:12.326779: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [9]:
import librosa


def analyze_voice_boldness():
    # Load the audio file using librosa
    audio_path = '/home/jegathees5555/Documents/recruitz/backend/audio/output.mp3'
    y, sr = librosa.load(audio_path)

    # Compute the boldness score (loudness)
    boldness_score = np.mean(np.abs(y))

    # Compute the clarity score
    # y_trimmed, _ = librosa.effects.trim(y)
    # stft = np.abs(librosa.stft(y_trimmed))
    # clarity_score = np.mean(stft) 
    # clarity_score = clarity_score * 94/100

    # return  clarity_score * 94/100
    return boldness_score * 1000

In [10]:
def analyze_voice_clarity():
    # Load the audio file using librosa
    audio_path = '/home/jegathees5555/Documents/recruitz/backend/audio/output.mp3'
    y, sr = librosa.load(audio_path)

    # Compute the boldness score (loudness)
    boldness_score = np.mean(np.abs(y))

    # Compute the clarity score
    y_trimmed, _ = librosa.effects.trim(y)
    stft = np.abs(librosa.stft(y_trimmed))
    clarity_score = np.mean(stft) 
    clarity_score = clarity_score * 94/100 * 100

    return  clarity_score

In [11]:
@app.route('/get_result', methods=['GET'])
def getResult():
    try:
        video_path = '/home/jegathees5555/Documents/recruitz/backend/video/uploaded_video.webm'
        eye_contact = eye_tracking.eye_tracking_method(video_path)
        voice.voice_extraction_main()
        
        # Call the functions to obtain the values
        clarity = int(analyze_voice_clarity())
        boldness = int(analyze_voice_boldness())
        
        # Calculate confidence and overall scores
        confidence = int(analyze_boldness_and_confidence())
        overall = (eye_contact + boldness + clarity + confidence) // 4
        
        # Create the JSON response within the application context
        with app.app_context():
            return jsonify({
                "eye_contact": eye_contact,
                "boldness": boldness,
                "clarity": clarity,
                "confidence": confidence,
                "overall": overall
            })
    except Exception as e:
        return "Error occurred: " + str(e), 500


In [12]:
# if __name__ == '__main__':
#     # Start the Flask app
#     app.run()

In [13]:
if __name__ == '__main__':
    # Set up ngrok and expose the Flask app
    # ngrok_tunnel = ngrok.connect(5000)
    # print('Public URL:', ngrok_tunnel.public_url)
    app.run(host='localhost', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5000
Press CTRL+C to quit


/home/jegathees5555/Documents/recruitz/backend/video/uploaded_video.webm
uploaded_video.webm


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

80.25316571788993
1/1 [==============================] - 0s 14ms/step


...

[matroska,webm @ 0x7fd46802a700] Length 5 indicated by an EBML number's first byte 0x0b at pos 510283 (0x7c94b) exceeds max length 4.
[NULL @ 0x7fd46802f600] Invalid profile 5.
[vp8 @ 0x7fd468068380] Header size larger than data provided
[NULL @ 0x7fd468064180] Error parsing Opus packet header.
[NULL @ 0x7fd468064180] Error parsing Opus packet header.
[NULL @ 0x7fd46802f600] Invalid profile 5.
[vp8 @ 0x7fd46809adc0] Header size larger than data provided
[NULL @ 0x7fd468064180] Error parsing Opus packet header.
[NULL @ 0x7fd468064180] Error parsing Opus packet header.
[NULL @ 0x7fd46802f600] Invalid profile 5.
[vp8 @ 0x7fd468034e80] Header size larger than data provided
[NULL @ 0x7fd468064180] Error parsing Opus packet header.
[NULL @ 0x7fd468064180] Error parsing Opus packet header.
[NULL @ 0x7fd46802f600] Invalid profile 5.
[vp8 @ 0x7fd468056f80] Header size larger than data provided
[NULL @ 0x7fd468064180] Error parsing Opus packet header.
[NULL @ 0x7fd46802f600] Invalid profile 5.
[

1/1 [==============================] - 0s 20ms/step


127.0.0.1 - - [27/Aug/2023 15:08:21] "POST /upload_video_new HTTP/1.1" 200 -


65.53599438217525
1/1 [==============================] - 0s 14ms/step


127.0.0.1 - - [27/Aug/2023 15:09:14] "GET /get_result HTTP/1.1" 200 -
